# Assignment 3

# Visualization Technique (25%)


# A narrative description of the visualization you are planning to use, describing how it works

I am planning to use Bar and Scatter Charts and heatmap via altair.
Scatter plot is best when both x and y are numeric, bar is appropriate when one variable is a numeric and the other is categorical.Heatmap works good if one needs to check the distribution across 2 categorical variables.


# A discussion of in which circumstances this visualization should and should not be used (what is it close to? What else could you consider? How does it relate to specific aspects of data?

Scatter plot - Use it when you want to check the relation between continuous variables and/or see different clusters in the data.  This will fail if one of the variable is not continuous
Bar plot - Use it when you want to check the relation between a continuous and a categorical variable.  This will fail if both the vaiables are categorical.
Heat Map - Use it when you want to check the relation between 2 categorical variables.  This will fail if you want to check the different clusters and their proximities. Although facet functionality of altair can help make heat map for different clusters.

# Visualization Library (25%)
# The library you are going to use, and a background on why the library is good for this visualization. For instance, who created it? Is it open source? How do you install it?

I am using altair for the visualization, I was introduced to altair in one of the MADS courses and I like how easily we can create charts.

It is an open source library developed by Jake Vanderplas and Brian Granger.

Altair can be installed, along with the example datasets in vega_datasets, using:

$ pip install altair vega_datasets

Reference Link : https://altair-viz.github.io/getting_started/installation.html


# A discussion of the general approach and limitations of this library. For instance, Is it declarative or procedural? Does it integrate with Jupyter? Why did you decide to use this library (especially if there are other options)?

Altair is not good if we are interested in large data sets. However for simple datasets altair makes different chart types very easy.

It is declarative, in the sense you have to tell the library what needs to be done and it will figure out the how part.

Yes, the library integrates with Jupyter and one can also generate html and interactive charts very easily. 

Easy implementation of interactivity is a very strong benefit of using altair. Interaction will help to find patterns in the data.

# Demonstration (50%)
The dataset you picked and instructions for cleaning the dataset. You should pick a suitable dataset to demonstrate the technique, toolkit, and problem you are facing. 
The quality of your demonstration. First demonstrate the basics of this approach, then show a few of the edges of how the library might be used for other cases. This is the "meat" of the assignment.





In this assignment I am using the Mall_Customer.csv that is used in one of the sample assignment3_examples

Ref https://www.kaggle.com/shwetabh123/mall-customers

In [1]:
#Importing libraries

import pandas as pd
import numpy as np
import altair as alt

In [2]:
df = pd.read_csv("Mall_Customers.csv")

I am using the same threshold as used in the sample example for the Income and Expenditure categories:

Below are the different thresholds I have used


Income Categories: Low for <40, Medium >=40 and <70, High >=70

Spending Categories: Low <40, Medium >=40 and <60, High >=60

Making different Income and Expediture Categories 

In [3]:
df['Income_Category'] = np.where(df['Annual Income (k$)']>=70,"High Income", np.where(df['Annual Income (k$)']>=40,"Medium Income","Low Income"))
df['Spend_Category'] = np.where(df['Spending Score (1-100)']>=60,"High Spend", np.where(df['Spending Score (1-100)']>=40,"Medium Spend","Low Spend"))

In [4]:
# Checking out how many Customer comes under the different Income and Expenditure Categories
df1 = pd.DataFrame(df.groupby(['Income_Category'])['CustomerID'].count()).reset_index()
df2 = pd.DataFrame(df.groupby(['Spend_Category'])['CustomerID'].count()).reset_index()

In [5]:
c1 = alt.Chart(df1).mark_bar().properties(title='Income Category Distribution',height=200, width=300).encode(
    x = alt.X('CustomerID', title='Count of Customers'),
    y = alt.Y('Income_Category',title = 'Income Category', sort = ['High Income','Medium Income','Low Income'])
)

c2 = alt.Chart(df2).mark_bar().properties(title='Spend Category Distribution',height=200, width=300).encode(
    x = alt.X('CustomerID', title='Count of Customers'),
    y = alt.Y('Spend_Category',title = 'Spend Category', sort = ['High Spend','Medium Spend','Low Spend'])
)

c1 | c2

alt.HConcatChart(...)

As can be seen in above Charts most of the Customers are Medium Income and Medium Spend.

Lets check if the Income/Spend has different patters for the Gender

In [6]:
alt.Chart(df).mark_circle().encode(
    x = 'Age:Q',
    y = 'Annual Income (k$):Q',
    facet = 'Gender:N',
    color = 'Spend_Category:N'
)

alt.Chart(...)

Looks like the Income goes up with Age till age reaches 45-50, beyond this the Income starts going down.

In [7]:
alt.Chart(df).mark_circle().encode(
    x = 'Age:Q',
    y = 'Spending Score (1-100):Q',
    facet = 'Gender:N',
    color = 'Income_Category:N'
)

alt.Chart(...)

Trend is a bit different for Spend, even younger people Spend good money and expenditure is high till 50 yrs of age, post that there seems to be a sudden decline in the Spend

Transforming the numeric ages to different age bins, these Age Bins will be used later in the analysis

In [8]:
bin_val = [0,20,40,50,60,120]
bin_lab = ['<20Yrs','20-40Yrs','40-50Yrs','50-60Yrs','>60Yrs']
df['Age_Bin'] = pd.cut(df['Age'],bin_val, labels = bin_lab)

Let us see the distribution of the Customers across different Income and Spend Categories

In [9]:
df3 = pd.pivot_table(df, values ='CustomerID', index ='Income_Category',
                         columns ='Spend_Category', aggfunc = 'count').unstack().reset_index()
df3 = df3.rename(columns = {0:'Customer Count'})
df3.head()

,Spend_Category,Income_Category,Customer Count
0,High Spend,High Income,38.0
1,High Spend,Low Income,23.0
2,High Spend,Medium Income,4.0
3,Low Spend,High Income,37.0
4,Low Spend,Low Income,22.0


In [10]:
c1 = alt.Chart(df3).mark_rect().properties(title='Income Vs Spend Distribution (# Customers)',height=500, width=700).encode(
alt.X('Income_Category:O',sort = ['High Income','Medium Income','Low Income']),
    alt.Y('Spend_Category:O',sort = ['High Spend','Medium Spend','Low Spend']),
    color = 'Customer Count:Q',
)


c2 = alt.Chart(df3).mark_text(size = 25).properties(title='Income Vs Spend Distribution (# Customers)',height=500, width=700).encode(
alt.X('Income_Category:O',sort = ['High Income','Medium Income','Low Income']),
    alt.Y('Spend_Category:O',sort = ['High Spend','Medium Spend','Low Spend']),
    text = alt.Text('Customer Count:Q'),
)

c1+c2

alt.LayerChart(...)

As can be seen in the above heat map that there are 38 Customers, who are both High Income and High Spend, these could be best Customers to target to and can be also called as the Premium Customers

There are 23 Customers, who are High Spend and Low Income and these may be risky Customers as they may be spending more than what they can afford and these are the risky customers and may be tagged as Careless.

There are 37 + 22 = 59 Customers, who Spend Low and can be called as Frugal Customers

We can classify rest of the Customers as Economical as they have Medium Spend

In [11]:
# Making different Customer Types
Premium = (df['Income_Category'] == "High Income") & (df['Spend_Category'] == "High Spend")
Frugal = df['Spend_Category'] == "Low Spend"
Careless = (df['Income_Category'].isin(["Medium Income","Low_Income"])) & (df['Spend_Category'] == "High Spend")

df['Customer Type'] = np.where(Premium,'Premium',np.where(Frugal,"Frugal",np.where(Careless,"Careless","Economical")))

In [12]:
# Distribution of Female Customers across different Types
female = pd.DataFrame(100 *df[df['Gender'] == 'Female'].groupby('Customer Type')['CustomerID'].count()/df[df['Gender'] == 'Female'].groupby('Customer Type')['CustomerID'].count().sum()).reset_index()

In [13]:
# Distribution of Male Customers across different Types
male = pd.DataFrame(100 *df[df['Gender'] == 'Male'].groupby('Customer Type')['CustomerID'].count()/df[df['Gender'] == 'Male'].groupby('Customer Type')['CustomerID'].count().sum()).reset_index()

In [14]:
c1 = alt.Chart(male).mark_bar().properties(title='Customer Type Distribution for Male in %',height=300, width=400).encode(
    x = alt.X('Customer Type:O', title='Customer Type', sort = ['Premium','Economical','Frugal','Careless']),
    y = alt.Y('CustomerID:Q',title = '% Distribution')
)

c2 = alt.Chart(female).mark_bar().properties(title='Customer Type Distribution for Female in %',height=300, width=400).encode(
    x = alt.X('Customer Type:O', title='Customer Type', sort = ['Premium','Economical','Frugal','Careless']),
    y = alt.Y('CustomerID:Q',title = '% Distribution')
)

c1 | c2

alt.HConcatChart(...)

As can be seen in the Charts above, Female has lesser percentage of Careless Customers, although Male on the other has have slightly higher Frugal percentage

Now lets see how different Age Bins and Customer Type interact

In [15]:
age_band = df.groupby(['Customer Type','Age_Bin'])['CustomerID'].count().reset_index()

In [16]:
c1 = alt.Chart(age_band).mark_rect().properties(title='Income Vs Spend Distribution',height=500, width=700).encode(
alt.X('Customer Type:O',sort = ['Premium','Economical','Frugal','Careless']),
    alt.Y('Age_Bin:O', sort = ['<20Yrs', '20-40Yrs', '40-50Yrs', '50-60Yrs', '>60Yrs']),
    color = 'CustomerID:Q',
)

c1

alt.Chart(...)

As can be seen highest Customers are in the 20-40Yrs and Economical band and may be a good target for the Economical items. However there are also many 20-40 Yrs Customers who are Premium, these may be the guys people who are earning well and have a long career ahead to save enough.
Interestingly the highest contibution of Careless also comes from 20-40 Yrs band and these Customers may be less preferred.

Lets see how the trend looks in terms of Percentage

In [17]:
age_band2 = pd.DataFrame(100*df.groupby(['Customer Type','Age_Bin'])['CustomerID'].count()/df.groupby(['Customer Type','Age_Bin'])['CustomerID'].count().sum()).reset_index()

In [18]:
c1 = alt.Chart(age_band2).mark_rect().properties(title='Income Vs Spend Distribution in Percentage',height=500, width=700).encode(
alt.X('Customer Type:O',sort = ['Premium','Economical','Frugal','Careless']),
    alt.Y('Age_Bin:O', sort = ['<20Yrs', '20-40Yrs', '40-50Yrs', '50-60Yrs', '>60Yrs']),
    color = 'CustomerID:Q',
)

c1

alt.Chart(...)

Below top 5 combination of Customer Type and Age Bin constitutes about 70% of the Customers

In [19]:
age_band2.sort_values('CustomerID', ascending = False).head(5)

,Customer Type,Age_Bin,CustomerID
6,Economical,20-40Yrs,22.5
16,Premium,20-40Yrs,19.0
12,Frugal,40-50Yrs,10.0
11,Frugal,20-40Yrs,9.5
7,Economical,40-50Yrs,9.0


In [20]:
alt.Chart(df).mark_circle().properties(title = "Customer Type across different Age Bins", height=150, width=300).encode(
    x = 'Spending Score (1-100):Q',
    y = 'Annual Income (k$):Q',
    color = 'Customer Type:N'
).facet(
    facet='Age_Bin:O',
    columns=2
)

alt.FacetChart(...)

As we can see, all age bands other than the 20-40 Yrs comprises of mostly Frugal and Economical.20-40 Yrs band is the most diverse.